In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:

import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns;sns.set()

from sklearn.linear_model    import LogisticRegression
from sklearn.tree            import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing   import StandardScaler
from sklearn.metrics         import confusion_matrix,classification_report,accuracy_score,roc_auc_score,roc_curve

from imblearn.under_sampling import RandomUnderSampler

from IPython.display         import Image
from sklearn.tree            import export_graphviz


%matplotlib inline

In [ ]:
data = pd.read_csv(r"../input/creditcardfraud/creditcard.csv")
data.head()

In [ ]:
data.describe()

check missing values
****

In [ ]:
data.isnull().sum().max()

In [ ]:
fig,ax  = plt.subplots(figsize = (6,4))
ax = sns.countplot(x = 'Class',data= data)
plt.tight_layout()

In [ ]:
print(data['Class'].value_counts())
print('\n percentage of fraud:{:.2%}'.format((data[data['Class']==1].shape[0]/data.shape[0])))

o.17% of data is fraud 
dataset is highly imbalanced


In [ ]:
corr = data.corr()
fig, ax = plt.subplots(figsize=(9, 7))

sns.heatmap(corr, xticklabels=corr.columns, yticklabels=corr.columns,
            linewidths=.1, cmap="RdBu", ax=ax)
plt.tight_layout()

no relation between variables


preprocessing

scaling of data

In [ ]:
data_clean = data.copy()
scaler = StandardScaler()
data_clean['std_amount'] = scaler.fit_transform(data_clean['Amount'].values.reshape(-1, 1))
data_clean['std_time'] = scaler.fit_transform(data_clean['Time'].values.reshape(-1, 1))

data_clean.drop(['Amount', 'Time'], axis=1, inplace=True)

data_clean.head()

In [ ]:
# define our feature matrix and target vector
X = data_clean.drop('Class', axis=1)
y = data_clean['Class']

# train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y)

undersampling

In [ ]:
rus = RandomUnderSampler()
X_rus,y_rus = rus.fit_sample(X_train,y_train)

#lets check sampled data
print(pd.Series(y_rus).value_counts())

In [ ]:
#new_correlation
cor_rus = pd.DataFrame(X_rus).corr()

fig,ax = plt.subplots(figsize = (9,7))
sns.heatmap(cor_rus,xticklabels=corr.columns, yticklabels=corr.columns,
            linewidths=.1, cmap="RdBu", ax=ax)
plt.tight_layout()

**LogisticRegression**

In [ ]:
model_log = LogisticRegression()

model_log.fit(X_rus,y_rus)

y_pred_log = model_log.predict(X_test)

In [ ]:
# classification report
print(classification_report(y_test, y_pred_log))

# ROC AUC score
print("AUC: {:.2f}\n".format(roc_auc_score(y_test, y_pred_log)))

# confusion matrix
fig, ax = plt.subplots()
sns.heatmap(confusion_matrix(y_test, y_pred_log, normalize='true'), annot=True, ax=ax)

ax.set_title("Confusion Matrix")
ax.set_ylabel("Real Value")
ax.set_xlabel("Predicted")

plt.show()

In [ ]:
model_tree = DecisionTreeClassifier(max_depth=4,criterion = 'entropy')
model_tree.fit(X_rus,y_rus)
y_pred_tree = model_tree.predict(X_test)
#clssi_reprt
print(classification_report(y_test,y_pred_tree))
#auc_roc_score
print("AUC: {:.2%}\n ".format(roc_auc_score(y_test,y_pred_tree)))


# confusion matrix
fig, ax = plt.subplots()
sns.heatmap(confusion_matrix(y_test, y_pred_tree, normalize='true'), annot=True, ax=ax)

ax.set_title("Confusion Matrix")
ax.set_ylabel("Real Value")
ax.set_xlabel("Predicted")

plt.show()      

**Conclusion**

Both models, **Logistic Regression** and **Decision Tree** performed extremely well in classifying credit card activities into the classes Regular Transaction and Fraudulent Transaction, with accuracy, ROC AUC scores and precision above 90%. Although they produced similar outcomes, the Logistic Regression showed slightly better results, with a greater ROC AUC score, which measures how well the model is capable to distinguish between classes.

> One of the biggest problems is the occurrence of False Positives, that is when the algorithm incorrectly detects a fraud. Thus, we are always searching for ways to shrink even more that 3% of False Positives.

In [ ]:
nan